In [1]:
from pathlib import Path

try:
    import yaml
except ImportError as exc:
    raise RuntimeError("PyYAML is required to load the dataset config. Install it with 'pip install pyyaml'.") from exc

from ultralytics import YOLO

# Build a repo-relative path so the YAML resolves regardless of the working directory
if "__file__" in globals():
    repo_root = Path(__file__).resolve().parent.parent
else:
    repo_root = Path.cwd()
    if not (repo_root / "datasets").exists() and (repo_root.parent / "datasets").exists():
        repo_root = repo_root.parent

data_yaml = repo_root / "datasets" / "VG150" / "YOLO_anno" / "train_ubuntu.yaml"
if not data_yaml.exists():
    raise FileNotFoundError(f"Dataset YAML not found at {data_yaml}")

with data_yaml.open("r", encoding="utf-8") as f:
    raw_config = yaml.safe_load(f)

dataset_root = data_yaml.parent
resolved_config = {
    "path": str(dataset_root),
    "train": str(dataset_root / "train"),
    "val": str(dataset_root / "val"),
    "test": str(dataset_root / "test"),
}

for split_name in ("train", "val", "test"):
    split_dir = dataset_root / split_name
    if not split_dir.exists():
        raise FileNotFoundError(f"Expected dataset split folder missing: {split_dir}")

names = raw_config.get("names")
if names:
    resolved_config["names"] = names
    resolved_config["nc"] = len(names)


In [3]:
resolved_config

{'path': '/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno',
 'train': '/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/train',
 'val': '/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/val',
 'test': '/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/test',
 'names': ['bag',
  'basket',
  'bin',
  'blind',
  'book',
  'bookshelf',
  'bottle',
  'bowl',
  'box',
  'button',
  'cabinet',
  'camera',
  'can',
  'carpet',
  'ceiling',
  'chair',
  'clock',
  'computer',
  'couch',
  'counter',
  'cup',
  'curtain',
  'desk',
  'dishwasher',
  'door',
  'drawer',
  'eyeglasses',
  'faucet',
  'floor',
  'flower',
  'food',
  'frame',
  'fruit',
  'glass',
  'hand',
  'handle',
  'head',
  'jacket',
  'key',
  'keyboard',
  'knife',
  'knob',
  'lamp',
  'laptop',
  'leg',
  'lid',
  'light',
  'luggage',
  'magazine',
  'microwave',
  'mirror',
  'monitor',
  'mouse',
  'mug',
  'oven',
  'pan',
  'paper',
  'pen',
  'person',
  'phone',
  'pillow',
  'plant',
  'plate',
  'poster',
  

In [4]:
# Train YOLO backbone with corrected configuration
from ultralytics import YOLO
import os

# Ensure we're in the correct directory
os.chdir("/workspace/SGG-Benchmark")
print("Current directory:", os.getcwd())

# Use the corrected YAML file (fixed Windows/Linux path issue)
corrected_yaml_path = "/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/train_ubuntu.yaml"

# Verify the YAML file exists
if os.path.exists(corrected_yaml_path):
    print("✅ Using corrected YAML:", corrected_yaml_path)
else:
    print("❌ YAML file not found:", corrected_yaml_path)
    raise FileNotFoundError(f"YAML file not found: {corrected_yaml_path}")

# Initialize YOLO model
model = YOLO("../backbone/yolov8m.pt")  # Load pre-trained YOLOv8m

# Train the model with corrected configuration
print("🚀 Starting YOLO training...")
results = model.train(
    data=corrected_yaml_path,  # Use the fixed YAML
    epochs=5,
    imgsz=640,
    batch=16,  # Adjust based on GPU memory
    device=0,  # Use GPU 0
    workers=4,
    patience=50,
    save=True,
    verbose=True,
    project="runs/detect",  # Save in runs/detect folder
    name="yolo11n_vg150"     # Experiment name
)

print("✅ Training completed!")
print("Model saved to:", results.save_dir)

Current directory: /workspace/SGG-Benchmark
✅ Using corrected YAML: /workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/train_ubuntu.yaml
🚀 Starting YOLO training...
New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.100 🚀 Python-3.11.14 torch-2.2.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48670MiB)


engine/trainer: task=detect, mode=train, model=../backbone/yolov8m.pt, data=/workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/train_ubuntu.yaml, epochs=5, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=runs/detect, name=yolo11n_vg1503, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

/root/miniconda3/envs/sgg_benchmark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AMP: checks passed ✅


train: Scanning /workspace/SGG-Benchmark/datasets/VG150/YOLO_anno/train/labels... 61359 images, 0 backgrounds, 0 corrupt: 100%|██████████| 61359/61359 [00:37<00:00, 1643.52it/s]

...

                   all      13148      91722      0.331      0.339      0.261      0.144

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      10.8G      1.433      1.981      1.548        213        640: 100%|██████████| 3835/3835 [10:35<00:00,  6.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 411/411 [00:59<00:00,  6.96it/s]


                   all      13148      91722      0.341      0.348      0.271      0.152

5 epochs completed in 0.982 hours.
Optimizer stripped from runs/detect/yolo11n_vg1503/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/yolo11n_vg1503/weights/best.pt, 52.1MB

Validating runs/detect/yolo11n_vg1503/weights/best.pt...
Ultralytics 8.3.100 🚀 Python-3.11.14 torch-2.2.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
Model summary (fused): 92 layers, 25,888,396 parameters, 0 gradients, 79.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 411/411 [01:00<00:00,  6.83it/s]


                   all      13148      91722      0.341      0.348      0.271      0.152
                   bag        994       1413      0.301      0.347       0.24      0.123
                basket        284        383      0.276      0.269      0.184     0.0921
                   bin        242        281      0.338      0.363      0.247      0.143
                 blind        121        157      0.302      0.255      0.178      0.105
                  book        370        724      0.286      0.199      0.157     0.0772
             bookshelf         78         96      0.331      0.391      0.297      0.151
                bottle        674       1104      0.354       0.47       0.34      0.177
                  bowl        565        845      0.396      0.454      0.384      0.245
                   box        587        854      0.312      0.196      0.164     0.0962
                button        218        396      0.283     0.0758     0.0781     0.0376
               cabine